<a href="https://colab.research.google.com/github/birusolankar/Pyspark-Bigdata/blob/main/window_functions_25_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pyspark.sql import SparkSession


In [4]:
spark = SparkSession.builder.appName("window_function").getOrCreate()

In [5]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )

columns = ['Employee_name', 'department', 'salary']

df = spark.createDataFrame(data = simpleData, schema = columns)

In [6]:
df.show()

+-------------+----------+------+
|Employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [9]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import col

In [11]:
df.withColumn('dense_rank', f.dense_rank().over(Window.partitionBy(col('department')).orderBy(col('salary').desc()))).show()

+-------------+----------+------+----------+
|Employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|          Jen|   Finance|  3900|         1|
|        Scott|   Finance|  3300|         2|
|        Maria|   Finance|  3000|         3|
|         Jeff| Marketing|  3000|         1|
|        Kumar| Marketing|  2000|         2|
|      Michael|     Sales|  4600|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|        James|     Sales|  3000|         3|
|        James|     Sales|  3000|         3|
+-------------+----------+------+----------+



In [12]:
df.withColumn('dense_rank', f.dense_rank().over(Window.partitionBy(col('department')).orderBy(col('salary').desc()))) \
.filter(col('dense_rank') == 1).show()

+-------------+----------+------+----------+
|Employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|          Jen|   Finance|  3900|         1|
|         Jeff| Marketing|  3000|         1|
|      Michael|     Sales|  4600|         1|
+-------------+----------+------+----------+



In [13]:
df.withColumn('dense_rank', f.dense_rank().over(Window.partitionBy(col('department')).orderBy(col('salary').desc()))) \
.withColumn('rank', f.rank().over(Window.partitionBy(col('department')).orderBy(col('salary').desc()))) \
.withColumn('row_number', f.row_number().over(Window.partitionBy(col('department')).orderBy(col('salary').desc()))) \
.show()

+-------------+----------+------+----------+----+----------+
|Employee_name|department|salary|dense_rank|rank|row_number|
+-------------+----------+------+----------+----+----------+
|          Jen|   Finance|  3900|         1|   1|         1|
|        Scott|   Finance|  3300|         2|   2|         2|
|        Maria|   Finance|  3000|         3|   3|         3|
|         Jeff| Marketing|  3000|         1|   1|         1|
|        Kumar| Marketing|  2000|         2|   2|         2|
|      Michael|     Sales|  4600|         1|   1|         1|
|       Robert|     Sales|  4100|         2|   2|         2|
|         Saif|     Sales|  4100|         2|   2|         3|
|        James|     Sales|  3000|         3|   4|         4|
|        James|     Sales|  3000|         3|   4|         5|
+-------------+----------+------+----------+----+----------+



In [16]:
df.withColumn('lag', f.lag('salary', 1).over(Window.partitionBy(col('department')).orderBy('salary'))).show()

+-------------+----------+------+----+
|Employee_name|department|salary| lag|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|NULL|
|        Scott|   Finance|  3300|3000|
|          Jen|   Finance|  3900|3300|
|        Kumar| Marketing|  2000|NULL|
|         Jeff| Marketing|  3000|2000|
|        James|     Sales|  3000|NULL|
|        James|     Sales|  3000|3000|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|4100|
|      Michael|     Sales|  4600|4100|
+-------------+----------+------+----+



In [17]:
df.withColumn('lag', f.lead('salary', 1).over(Window.partitionBy(col('department')).orderBy('salary'))).show()

+-------------+----------+------+----+
|Employee_name|department|salary| lag|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|3300|
|        Scott|   Finance|  3300|3900|
|          Jen|   Finance|  3900|NULL|
|        Kumar| Marketing|  2000|3000|
|         Jeff| Marketing|  3000|NULL|
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4100|
|         Saif|     Sales|  4100|4600|
|      Michael|     Sales|  4600|NULL|
+-------------+----------+------+----+

